In [1]:
import pandas as pd
import re
import unicodedata

In [2]:
df = pd.read_excel("../data/Talent_Academy_Case_DT_2025.xlsx")
df.head()

,HastaNo,Yas,Cinsiyet,KanGrubu,Uyruk,KronikHastalik,Bolum,Alerji,Tanilar,TedaviAdi,TedaviSuresi,UygulamaYerleri,UygulamaSuresi
0,145134,60,Kadın,0 Rh+,Türkiye,"Becker Musküler Distrofisi, Hiportiroidizm, Ka...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",TOZ,Ayak bileği ve ayağın yüzeysel yaralanması,Ayak Bileği,5 Seans,Ayak Bileği,20 Dakika
1,145135,28,Erkek,0 Rh+,Türkiye,"Duchenne Musküler Distrofisi, Myastenia gravis...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",NaN,"Omuzun darbe sendromu,DORSALJİ, DİĞER, SERVİKO...",Dorsalji -Boyun+trapez+skapular,15 Seans,Boyun,20 Dakika
2,145135,28,Erkek,0 Rh+,Türkiye,"Duchenne Musküler Distrofisi, Myastenia gravis...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",NaN,"Omuzun darbe sendromu,DORSALJİ, DİĞER, SERVİKO...",Dorsalji -Boyun+trapez+skapular,15 Seans,"Boyun,Sırt",20 Dakika
3,145135,28,Erkek,0 Rh+,Türkiye,"Duchenne Musküler Distrofisi, Myastenia gravis...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",NaN,"Omuzun darbe sendromu,DORSALJİ, DİĞER, SERVİKO...",Dorsalji -Boyun+trapez+skapular,15 Seans,Boyun,5 Dakika
4,145135,28,Erkek,0 Rh+,Türkiye,"Duchenne Musküler Distrofisi, Myastenia gravis...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",NaN,"Omuzun darbe sendromu,DORSALJİ, DİĞER, SERVİKO...",Dorsalji -Boyun+trapez+skapular,15 Seans,"Boyun,Sırt",20 Dakika


## 1. Eksik Değer İşlemleri
### 1.1 Sayısal Sütunlar
- **Yas**, **TedaviSuresi** ve **UygulamaSuresi** sütunlarında eksik değer bulunmadığı için herhangi bir işlem yapılmamıştır.

In [3]:
df.isnull().sum().sort_values()

HastaNo              0
Yas                  0
Uyruk                0
TedaviAdi            0
TedaviSuresi         0
UygulamaSuresi       0
Bolum               11
Tanilar             75
Cinsiyet           169
UygulamaYerleri    221
KronikHastalik     611
KanGrubu           675
Alerji             944
dtype: int64

### 1.2 Kategorik Sütunlar

In [4]:
df["KanGrubu"] = df["KanGrubu"].fillna("Bilinmiyor")
df["UygulamaYerleri"] = df["UygulamaYerleri"].fillna("Bilinmiyor")


In [5]:
df["Bolum"] = df["Bolum"].fillna(df["Bolum"].mode()[0])
df["Cinsiyet"] = df["Cinsiyet"].fillna(df["Cinsiyet"].mode()[0])

In [6]:
df.isnull().sum().sort_values()

HastaNo              0
Yas                  0
Cinsiyet             0
KanGrubu             0
Uyruk                0
Bolum                0
TedaviAdi            0
TedaviSuresi         0
UygulamaYerleri      0
UygulamaSuresi       0
Tanilar             75
KronikHastalik     611
Alerji             944
dtype: int64

### 1.3 Multi-label Eksik Değer İşlemleri

In [7]:
multi_label_cols = ["KronikHastalik", "Tanilar", "Alerji"]

for col in multi_label_cols:
    df[col] = df[col].fillna("yok")

df[multi_label_cols].isnull().sum()

KronikHastalik    0
Tanilar           0
Alerji            0
dtype: int64

## 2. Kategorik Değişkenlerde Temizlik

In [8]:
def clean_col(text):
    if pd.isnull(text):
        return text
    
    text = str(text).strip()
    
    text = text.lower()
    text = re.sub(r"\s+", " ", text)      
    text = re.sub(r"\s*-\s*", "-", text)  
    text = re.sub(r"\s*\+\s*", "+", text)
    text = re.sub(r",$", "", text)
    text = re.sub(r"-$", "", text)
    text = re.sub(r"$", "", text)
    
    text = text.title()
    text = unicodedata.normalize("NFC", text)
    return text

In [9]:
df["TedaviAdi"] = df["TedaviAdi"].apply(clean_col)
df["UygulamaYerleri"] = df["UygulamaYerleri"].apply(clean_col)


In [10]:
TedaviAdi_unique = sorted(df["TedaviAdi"].unique())
for val in TedaviAdi_unique:
    print(val)

print("Toplam benzersiz kategori:", len(TedaviAdi_unique))


1. Parmak Bağ Ve Volar Plak Yaralanması
1. Parmak Cmc Eklem Ağrısı
5. Metatars Fissürü+Bağ Lezyonu
Aksiller Web Sendromu+Kontraktür
Algonörodistrofi Sol Ayak
Algonörodistrofi Sol El
Alt Ekstremite Atrofi
Alt Ekstremite Atrofi+Yürüme
Alt Ekstremite Atrofi-Bilateral
Artroskopi Diz Tam Yükle Mobilizasyon
Ataksi Reh
Ayak Bileği
Ayak Bileği Distorsiyonu
Ayak Bileği Distorsiyonu-1
Ayak Bileği Distorsiyonu-2
Ayak Bileği Kontraktürü
Ayak Bileği Post Op
Aşil Ameliyatı Erken
Aşil Kısalığı
Aşil Rüptürü Op
Aşil Rüptürü Rehabilitasyonu
Aşil Rüptürü-1
Aşil Rüptürü-2
Aşil Tamiri
Aşil Tendiniti Sol-1
Aşil Tendiniti Sol-2
Baker Kisti Rüptürü
Bel
Bel Ftr
Beyin Kanaması+Atrofi
Bialetarl Diz İMplantı-Erken Rehabilitasyon
Bilarteral Implant-Erken Rehabilitasyon
Bilateral Diz Eklem İMplantı
Boyun
Boyun-Düşük Kol
Boyun-Sırt Ağrısı
Boyun-Trapezz
Deneme
Dirsek Eklem Çıkığı+Kontraktürü
Diz Ağrısı
Diz Implantı
Diz Menisküs
Diz Menisküs Erken Mobilizasyon
Diz Op-Erken Rehabilitasyon
Di̇Z Protezi̇ Sonrasi Tam Yük


In [11]:
replace_dict = {
    "Xx": "Bilinmiyor",
    "Deneme": "Bilinmiyor",
    "Onur": "Bilinmiyor",
    "Eklem Ağrsıı": "Eklem Ağrısı",         
    "Sol Omuz İMpingement": "Sol Omuz Impingement",
    "Sol Omuz İMpingemen": "Sol Omuz Impingement",
    "Sol Omuz İMpingement": "Sol Omuz Impingement", 
    "Sol Kalça İMplanti": "Sol Kalça Implantı",
    "Sağ Omuz İMpi̇Ngement": "Sağ Omuz İmpingement",
    "Sağ Omuz İMpingement": "Sağ Omuz İmpingement",
    "Tfcc Rehabilitasyon": "Tfcc Rehabilitasyonu",
    "El Rehabi̇Li̇Tasyonu": "El Rehabilitasyonu",
}


In [12]:
df["TedaviAdi"] = df["TedaviAdi"].replace(replace_dict)


In [13]:
TedaviAdi_unique = sorted(df["TedaviAdi"].unique())
for val in TedaviAdi_unique:
    print(val)

print("Toplam benzersiz kategori:", len(TedaviAdi_unique))

1. Parmak Bağ Ve Volar Plak Yaralanması
1. Parmak Cmc Eklem Ağrısı
5. Metatars Fissürü+Bağ Lezyonu
Aksiller Web Sendromu+Kontraktür
Algonörodistrofi Sol Ayak
Algonörodistrofi Sol El
Alt Ekstremite Atrofi
Alt Ekstremite Atrofi+Yürüme
Alt Ekstremite Atrofi-Bilateral
Artroskopi Diz Tam Yükle Mobilizasyon
Ataksi Reh
Ayak Bileği
Ayak Bileği Distorsiyonu
Ayak Bileği Distorsiyonu-1
Ayak Bileği Distorsiyonu-2
Ayak Bileği Kontraktürü
Ayak Bileği Post Op
Aşil Ameliyatı Erken
Aşil Kısalığı
Aşil Rüptürü Op
Aşil Rüptürü Rehabilitasyonu
Aşil Rüptürü-1
Aşil Rüptürü-2
Aşil Tamiri
Aşil Tendiniti Sol-1
Aşil Tendiniti Sol-2
Baker Kisti Rüptürü
Bel
Bel Ftr
Beyin Kanaması+Atrofi
Bialetarl Diz İMplantı-Erken Rehabilitasyon
Bilarteral Implant-Erken Rehabilitasyon
Bilateral Diz Eklem İMplantı
Bilinmiyor
Boyun
Boyun-Düşük Kol
Boyun-Sırt Ağrısı
Boyun-Trapezz
Dirsek Eklem Çıkığı+Kontraktürü
Diz Ağrısı
Diz Implantı
Diz Menisküs
Diz Menisküs Erken Mobilizasyon
Diz Op-Erken Rehabilitasyon
Di̇Z Protezi̇ Sonrasi Tam 

In [14]:
categorical_cols = ["Cinsiyet", "KanGrubu", "Uyruk", "Bolum"]

for col in categorical_cols:
    print(f"\n{col}")
    print(df[col].unique())
    print("Toplam benzersiz değer:", df[col].nunique())



Cinsiyet
['Kadın' 'Erkek']
Toplam benzersiz değer: 2

KanGrubu
['0 Rh+' 'A Rh+' 'B Rh+' 'B Rh-' 'AB Rh+' 'Bilinmiyor' 'A Rh-' '0 Rh-'
 'AB Rh-']
Toplam benzersiz değer: 9

Uyruk
['Türkiye' 'Tokelau' 'Azerbaycan' 'Libya' 'Arnavutluk']
Toplam benzersiz değer: 5

Bolum
['Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi' 'İç Hastalıkları'
 'Ortopedi Ve Travmatoloji' 'Kardiyoloji' 'Kalp Ve Damar Cerrahisi'
 'Genel Cerrahi' 'Tıbbi Onkoloji' 'Göğüs Hastalıkları' 'Nöroloji'
 'Laboratuar']
Toplam benzersiz değer: 10


## 3. Multi-label Sütunların Hazırlanması

In [15]:
multi_label_cols = ["Tanilar", "KronikHastalik", "Alerji", "UygulamaYerleri"]

for col in multi_label_cols:
    df[col] = df[col].apply(
        lambda x: [item.strip().lower() for item in str(x).split(",")]
    )

df[multi_label_cols].head()


,Tanilar,KronikHastalik,Alerji,UygulamaYerleri
0,[ayak bileği ve ayağın yüzeysel yaralanması],"[becker musküler distrofisi, hiportiroidizm, k...",[toz],[ayak bileği]
1,"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...","[duchenne musküler distrofisi, myastenia gravi...",[yok],[boyun]
2,"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...","[duchenne musküler distrofisi, myastenia gravi...",[yok],"[boyun, sırt]"
3,"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...","[duchenne musküler distrofisi, myastenia gravi...",[yok],[boyun]
4,"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...","[duchenne musküler distrofisi, myastenia gravi...",[yok],"[boyun, sırt]"


## 4. Rare Category Handling Islemi

In [16]:
def rare_category_handling_multilabel(df, col, threshold=10):
    freq = df[col].explode().value_counts()
    rare_cats = freq[freq < threshold].index
    
    df[col] = df[col].apply(
        lambda items: ["Nadir" if item in rare_cats else item for item in items]
    )
    return df

df = rare_category_handling_multilabel(df, "UygulamaYerleri", threshold=10)


In [17]:
print(df["UygulamaYerleri"].explode().value_counts().head(20))
print("Toplam kategori:", df["UygulamaYerleri"].explode().nunique())

UygulamaYerleri
bel                        543
boyun                      388
bilinmiyor                 221
diz                        177
sol omuz bölgesi           140
sağ omuz bölgesi           132
sırt                        96
sol el bilek bölgesi        86
sağ ayak bileği bölgesi     68
sol ayak bileği bölgesi     65
tüm vücut bölgesi           62
sol diz bölgesi             52
sağ el bilek bölgesi        49
sol kalça bölgesi           47
sağ kalça bölgesi           47
sağ diz bölgesi             32
kalça                       28
Nadir                       15
dirsek                      14
el bilek                    13
Name: count, dtype: int64
Toplam kategori: 23


In [18]:
def rare_category_handling(df, col, threshold=10):
    freq = df[col].value_counts()
    rare_cats = freq[freq < threshold].index
    
    df[col] = df[col].apply(lambda x: "Nadir" if x in rare_cats else x)
    return df


In [19]:
df = rare_category_handling(df, "TedaviAdi", threshold=10)

print(df["TedaviAdi"].value_counts().head(20))
print("Toplam kategori:", df["TedaviAdi"].nunique())


TedaviAdi
Nadir                              801
Dorsalji-Boyun+Trapez              231
İV Di̇Sk Bozukluğu-Bel             200
Dorsalji 1                         140
Dorsalji-Bel                       120
Sol Omuz Impingement               110
Gonartroz-Meniskopati               95
Sağ Omuz İmpingement                75
Boyun-Trapezz                       60
Dorsalji-Dorsal                     56
Alt Ekstremite Atrofi-Bilateral     37
Dorsalji Boyun 1                    35
Impingement Sağ                     22
Meni̇Sküs Tami̇Ri̇                  20
Sol Diz Implantı Reh                20
İV Di̇Sk Bozukluğu-Bel-2            18
El Rehabilitasyonu                  16
Skolyoz                             16
Patella Kırığı                      15
Bel Ftr                             15
Name: count, dtype: int64
Toplam kategori: 33


## 5. Encoding İşlemleri

In [20]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

categorical_cols = ["Cinsiyet", "KanGrubu", "Uyruk", "Bolum", "TedaviAdi"]

ohe = OneHotEncoder(sparse_output=False)

encoded = ohe.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded, 
                          columns=ohe.get_feature_names_out(categorical_cols), 
                          index=df.index)



In [21]:
from sklearn.preprocessing import MultiLabelBinarizer

multi_label_cols = ["Tanilar", "KronikHastalik", "Alerji", "UygulamaYerleri"]

df_temp = df.drop(columns=categorical_cols).copy()

for col in multi_label_cols:
    mlb = MultiLabelBinarizer()
    encoded = mlb.fit_transform(df_temp[col])
    mlb_df = pd.DataFrame(encoded, 
                          columns=[f"{col}_{cls}" for cls in mlb.classes_], 
                          index=df_temp.index)
    df_temp = pd.concat([df_temp.drop(columns=[col]), mlb_df], axis=1)


## 6. Sayısal Sütunların Ölçeklenmesi  

In [22]:
df["UygulamaSuresi"] = df["UygulamaSuresi"].str.replace(" Dakika", "").astype(int)
df["UygulamaSuresi"].head()

0    20
1    20
2    20
3     5
4    20
Name: UygulamaSuresi, dtype: int64

In [23]:
df_temp["UygulamaSuresi"] = df_temp["UygulamaSuresi"].str.replace(" Dakika", "").astype(int)

In [24]:
from sklearn.preprocessing import StandardScaler

numeric_cols = ["Yas", "UygulamaSuresi"]

scaler = StandardScaler()
df_temp[numeric_cols] = scaler.fit_transform(df_temp[numeric_cols])


In [25]:
df_encoded = pd.concat([df_temp.drop(columns=categorical_cols, errors="ignore"), 
                        encoded_df], axis=1)

print("Son dataframe boyutu:", df_encoded.shape)
print(df_encoded.head())


Son dataframe boyutu: (2235, 400)
   HastaNo       Yas TedaviSuresi  UygulamaSuresi  Tanilar_  \
0   145134  0.833459      5 Seans        0.546788         0   
1   145135 -1.271080     15 Seans        0.546788         0   
2   145135 -1.271080     15 Seans        0.546788         0   
3   145135 -1.271080     15 Seans       -1.846613         0   
4   145135 -1.271080     15 Seans        0.546788         0   

   Tanilar_'dorsalji̇  Tanilar_akromiyoklavikuler eklem burkulma ve gerilmesi  \
0                   0                                                  0        
1                   0                                                  0        
2                   0                                                  0        
3                   0                                                  0        
4                   0                                                  0        

   Tanilar_akut apandisit  Tanilar_akut peptik ülser  \
0                       0                   

In [26]:
df_encoded.to_csv("../data/preprocessed_data.csv", index=False, encoding="utf-8")

In [27]:
df_encoded.to_excel("../data/preprocessed_data.xlsx", index=False)

In [28]:
df_encoded

,HastaNo,Yas,TedaviSuresi,UygulamaSuresi,Tanilar_,Tanilar_'dorsalji̇,Tanilar_akromiyoklavikuler eklem burkulma ve gerilmesi,Tanilar_akut apandisit,Tanilar_akut peptik ülser,Tanilar_algonörodistrofi,...,TedaviAdi_Nadir,TedaviAdi_Patella Kırığı,TedaviAdi_Sağ Omuz İmpingement,TedaviAdi_Skolyoz,TedaviAdi_Sol Diz Implantı Reh,TedaviAdi_Sol Kalça Implantı,TedaviAdi_Sol Omuz Impingement,TedaviAdi_İMpingement Sağ,TedaviAdi_İV Di̇Sk Bozukluğu-Bel,TedaviAdi_İV Di̇Sk Bozukluğu-Bel-2
0,145134,0.833459,5 Seans,0.546788,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,145135,-1.271080,15 Seans,0.546788,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145135,-1.271080,15 Seans,0.546788,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,145135,-1.271080,15 Seans,-1.846613,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,145135,-1.271080,15 Seans,0.546788,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,145536,0.044257,15 Seans,-1.048813,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2231,145536,0.044257,15 Seans,0.546788,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2232,145537,-0.942246,15 Seans,0.546788,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2233,145537,-0.942246,15 Seans,-0.251013,0,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
df

,HastaNo,Yas,Cinsiyet,KanGrubu,Uyruk,KronikHastalik,Bolum,Alerji,Tanilar,TedaviAdi,TedaviSuresi,UygulamaYerleri,UygulamaSuresi
0,145134,60,Kadın,0 Rh+,Türkiye,"[becker musküler distrofisi, hiportiroidizm, k...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[toz],[ayak bileği ve ayağın yüzeysel yaralanması],Nadir,5 Seans,[Nadir],20
1,145135,28,Erkek,0 Rh+,Türkiye,"[duchenne musküler distrofisi, myastenia gravi...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[yok],"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...",Nadir,15 Seans,[boyun],20
2,145135,28,Erkek,0 Rh+,Türkiye,"[duchenne musküler distrofisi, myastenia gravi...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[yok],"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...",Nadir,15 Seans,"[boyun, sırt]",20
3,145135,28,Erkek,0 Rh+,Türkiye,"[duchenne musküler distrofisi, myastenia gravi...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[yok],"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...",Nadir,15 Seans,[boyun],5
4,145135,28,Erkek,0 Rh+,Türkiye,"[duchenne musküler distrofisi, myastenia gravi...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[yok],"[omuzun darbe sendromu, dorsalji̇, di̇ğer, ser...",Nadir,15 Seans,"[boyun, sırt]",20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,145536,48,Erkek,Bilinmiyor,Türkiye,"[aritmi, myastenia gravis, fascioscapulohumera...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi","[arveles, coraspin]",[eklemin kontraktürü],Ekstansör Tendon Rehabilitasyon,15 Seans,[sol el bilek bölgesi],10
2231,145536,48,Erkek,Bilinmiyor,Türkiye,"[aritmi, myastenia gravis, fascioscapulohumera...","Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi","[arveles, coraspin]",[eklemin kontraktürü],Ekstansör Tendon Rehabilitasyon,15 Seans,[sol el bilek bölgesi],20
2232,145537,33,Kadın,A Rh+,Türkiye,[yok],"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[sucuk],"[aşil tendon yaralanması, ​algonörodistrofi, ​...",Nadir,15 Seans,[sol ayak bileği bölgesi],20
2233,145537,33,Kadın,A Rh+,Türkiye,[yok],"Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi",[sucuk],"[aşil tendon yaralanması, algonörodistrofi, ay...",Nadir,15 Seans,[sol ayak bileği bölgesi],15
